# TEST TRAINED MODEL

## Import dependencies

In [1]:
import joblib
import pandas as pd
from ipaddress import ip_address
from rich.console import Console
from rich.table import Table

## Import model, encoders and scaler

In [2]:
knn_model = joblib.load("../trained_model/knn_model.joblib")
scaler = joblib.load("../trained_model/scaler.joblib")
detected_encoder = joblib.load("../trained_model/detected_encoder.joblib")
protocol_encoder = joblib.load("../trained_model/protocol_encoder.joblib")
severity_encoder= joblib.load("../trained_model/severity_encoder.joblib")

## Print result function

In [3]:
def print_results(attacks, solution_array):
    console = Console()
    table = Table()
    table.add_column("PREDICTED ATTACK", style="magenta")
    table.add_column("ATTACK", style="green")

    for attack, solution in zip(attacks, solution_array):
        table.add_row(attack, solution)

    console.print(table)

## Load Data

In [4]:
df_to_test = pd.read_csv("./test_df.csv")
df_to_test.head()

,IP,Protocol,DateTime,Interactions,Port,Severity,Response Time,Packet Size,Origin Country,Detected By
0,198.51.100.5,DNS,2025-02-12 10:46:52,397,53,High,1132,1017,Canada,IPS
1,172.16.5.3,UDP,2025-02-12 10:46:52,459,123,Critical,705,1488,France,IPS
2,10.10.10.10,HTTP,2025-02-12 10:46:52,89,80,Medium,138,222,UK,SIEM
3,192.168.100.10,ICMP,2025-02-12 10:46:52,89,0,Medium,615,1230,Japan,IPS
4,172.20.30.40,ICMP,2025-02-12 10:46:52,112,0,Critical,610,1223,China,IPS


In [5]:
df_solution = pd.read_csv("./test_df_solution.csv")
solution_array = df_solution.to_numpy().flatten()

## Process data

In [6]:
df_to_test.drop(columns=["DateTime", "Origin Country"], inplace=True)
df_to_test.head()

,IP,Protocol,Interactions,Port,Severity,Response Time,Packet Size,Detected By
0,198.51.100.5,DNS,397,53,High,1132,1017,IPS
1,172.16.5.3,UDP,459,123,Critical,705,1488,IPS
2,10.10.10.10,HTTP,89,80,Medium,138,222,SIEM
3,192.168.100.10,ICMP,89,0,Medium,615,1230,IPS
4,172.20.30.40,ICMP,112,0,Critical,610,1223,IPS


In [7]:
def convertir_ip(df):
    df['IP'] = df['IP'].apply(lambda x: int(ip_address(x)))
    return df

In [8]:
df_to_test = convertir_ip(df_to_test)
df_to_test.head()

,IP,Protocol,Interactions,Port,Severity,Response Time,Packet Size,Detected By
0,3325256709,DNS,397,53,High,1132,1017,IPS
1,2886731011,UDP,459,123,Critical,705,1488,IPS
2,168430090,HTTP,89,80,Medium,138,222,SIEM
3,3232261130,ICMP,89,0,Medium,615,1230,IPS
4,2886999592,ICMP,112,0,Critical,610,1223,IPS


In [9]:
df_to_test["Protocol"] = protocol_encoder.transform(df_to_test["Protocol"])
df_to_test["Severity"] = severity_encoder.transform(df_to_test["Severity"])
df_to_test["Detected By"] = detected_encoder.transform(df_to_test["Detected By"])


df_to_test.head()

,IP,Protocol,Interactions,Port,Severity,Response Time,Packet Size,Detected By
0,3325256709,0,397,53,1,1132,1017,3
1,2886731011,7,459,123,0,705,1488,3
2,168430090,2,89,80,2,138,222,4
3,3232261130,4,89,0,2,615,1230,3
4,2886999592,4,112,0,0,610,1223,3


In [10]:
df_to_test_scaled = scaler.transform(df_to_test)
df_to_test_processed = pd.DataFrame(df_to_test_scaled, columns=df_to_test.columns)
df_to_test_processed.head()


,IP,Protocol,Interactions,Port,Severity,Response Time,Packet Size,Detected By
0,0.764973,-1.531924,2.075082,-0.329898,0.071656,1.467177,0.532571,0.658329
1,0.449817,1.518980,2.612855,0.151171,-1.319721,0.576046,1.668755,0.658329
2,-1.503749,-0.660237,-0.596436,-0.144343,1.463033,-0.607259,-1.385192,1.401910
3,0.698140,0.211449,-0.596436,-0.694136,1.463033,0.388220,1.046386,0.658329
4,0.450010,0.211449,-0.396940,-0.694136,-1.319721,0.377785,1.029500,0.658329


## Predict Attacks

In [11]:
attacks = knn_model.predict(df_to_test_processed)
print_results(attacks, solution_array)

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ PREDICTED ATTACK     ┃ ATTACK               ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ DNS Amplification    │ DNS Amplification    │
│ DDoS                 │ DDoS                 │
│ Cross-Site Scripting │ Cross-Site Scripting │
│ Ping Flood           │ Ping Flood           │
│ DDoS                 │ DDoS                 │
│ DDoS                 │ DDoS                 │
│ SQL Injection        │ SQL Injection        │
│ Malware              │ Malware              │
│ Brute Force          │ Brute Force          │
│ DDoS                 │ DDoS                 │
│ Malware              │ Malware              │
│ DDoS                 │ DDoS                 │
│ DNS Spoofing         │ DNS Spoofing         │
│ DDoS                 │ DDoS                 │
│ DNS Amplification    │ DNS Amplification    │
│ Brute Force          │ Brute Force          │
└──────────────────────┴──────────────────────┘

In [12]:
# console = Console()

# # Crear la tabla con dos columnas
# table = Table()

# # Definir las columnas de la tabla
# table.add_column("PREDICTED ATTACK", style="magenta")
# table.add_column("ATTACK", style="green")

# # Agregar los datos de ambos arrays a la tabla
# for attack, solution in zip(attacks, solution_array):
#     table.add_row(attack, solution)

# # Imprimir la tabla en consola
# console.print(table)